In [ ]:
#AI Instructions

# Investment Strategy: dollar-cost-averaging, or buying into the market on a regular basis, regardless of fluctuations.

# Methodology:
# Simulate a dollar-cost-average strategy by investing $25 at the first trading day of each week.
# Calculate the week and use the first, or minimum date of that week as the first trading day.
# The first trading day will normally be Monday but there are exceptions for holidays.
# Assume the daily average price as the purchase price.  
# Calculate the daily average by using the opening and closing prices and averaging them.

# Using XLG as an example, if XLG opens at $49/share and closes at $51/share.
# I would have invested $25 at the average price of $50, meaning I would have purchased 0.5 shares @ $50.
# You will need to record a history of purchases for future steps.

# Next, we're going to continue calculating the return on that share.
# So each week, I buy a new share but my previous share may be worth more or less than my initial investment.

# We will eventually graph this
# The graph will show the money I've invested as one line and my growing, or declining, value of my shares on another line.


In [ ]:
import os
import yfinance as yf 
import pandas as pd
import openpyxl
import datetime as dt

# Ensure /data folder exists
output_folder = "data"
os.makedirs(output_folder, exist_ok=True)

etf_list = [
    "ALLY",
    "FBCG",
    "FMAG",
    "GGLL",
    "GOOGL",
    "HSBC",
    "MGK",
    "MSFT",
    "MSFU",
    "OEF",
    "QLD",
    "QQQ",
    "QQQM",
    "QQUP",
    "QQXL",
    "QTOP",
    "SPLG",
    "SPY",
    "SSO",
    "TQQQ",
    "TOPT",
    "UPRO",
    "VGT",
    "XLG"
]

for ticker_symbol in etf_list:
    ticker = yf.Ticker(ticker_symbol)
    data = ticker.history(period="10y", interval="1d")
    df = pd.DataFrame(data)
    df.drop(['Dividends', 'Stock Splits', 'Volume'], axis=1, inplace=True)
    df.insert(0, 'Date_add', df.index)
    df.insert(1, 'Symbol', ticker_symbol)
    df['week_of_year'] = df.index.isocalendar().week
    df['week_day'] = df.index.day_name()
    df['avg_daily_price'] = df[['Open', 'High', 'Low', 'Close']].mean(axis=1)
    df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')
    df['Date_add'] = pd.to_datetime(df['Date_add']).dt.strftime('%Y-%m-%d')
    csv_name = os.path.join(output_folder, f'{ticker_symbol.lower()}.csv')
    df.to_csv(csv_name, index=False)

In [ ]:
# This cell simulates a Dollar-Cost Averaging (DCA) strategy with monthly investments for each ETF in the list.
# - Investment Strategy: Invest $100 on the first trading day of each month in each ETF.
# - Methodology: For each month's first trading day, calculate the average price (mean of open and close), 
#   determine how many shares $100 buys, and track cumulative shares and invested amount.
# - Visualization: Plot cumulative invested amount vs. market value of holdings over time for each ETF.
# - Purpose: Compare monthly DCA results for all ETFs and observe periods of gain/loss.

investment_amount = 100

for ticker_symbol in etf_list:
    print(f"\nETF: {ticker_symbol}")
    csv_name = f'yf_{ticker_symbol.lower()}.csv'
    df = pd.read_csv(csv_name, parse_dates=['Date_add'])
    df.rename(columns={'Date_add': 'Date'}, inplace=True)
    df['Date'] = pd.to_datetime(df['Date'], utc=True).dt.tz_convert(None)

    # Filter for the first trading day of each month
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    first_days_df = df.groupby(['Year', 'Month']).first().reset_index()

    # Calculate daily average price
    first_days_df.loc[:, 'Avg Price'] = (first_days_df['Open'] + first_days_df['Close']) / 2

    # Simulate DCA purchases
    first_days_df.loc[:, 'Shares Purchased'] = investment_amount / first_days_df['Avg Price']
    first_days_df.loc[:, 'Cumulative Shares'] = first_days_df['Shares Purchased'].cumsum()

    # Calculate the market value of the cumulative shares
    first_days_df.loc[:, 'Market Value'] = first_days_df['Cumulative Shares'] * first_days_df['Close']

    # Calculate the cumulative invested amount
    first_days_df.loc[:, 'Cumulative Invested'] = investment_amount * (first_days_df.reset_index().index + 1)

    # Plot the DCA strategy results
    fig, ax = plt.subplots(figsize=(12, 8))

    ax.plot(first_days_df['Date'], first_days_df['Cumulative Invested'], label='Cumulative Invested', color='blue')
    ax.plot(first_days_df['Date'], first_days_df['Market Value'], label=f'Market Value ({ticker_symbol} Close)', color='green')
    ax.set_title(f'Dollar-Cost Averaging Strategy: Cumulative Invested Amount vs. Market Value Over Time ($100 Monthly) - {ticker_symbol}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Value ($)')
    ax.legend()
    ax.grid(True)

    plt.show()

    # Display the final few rows of the DataFrame for inspection
    # display(first_days_df.tail())

In [ ]:
# This cell simulates a Dollar-Cost Averaging (DCA) strategy with weekly investments for each ETF in the list.
# - Investment Strategy: Invest $25 every Tuesday in each ETF.
# - Methodology: For each Tuesday, calculate the average price (mean of open and close), 
#   determine how many shares $25 buys, and track cumulative shares and invested amount.
# - Visualization: Plot cumulative invested amount vs. market value of holdings over time for each ETF.
# - Purpose: Compare weekly DCA results for all ETFs and observe periods of gain/loss.

investment_amount = 25

for ticker_symbol in etf_list:
    print(f"\nETF: {ticker_symbol}")
    csv_name = f'yf_{ticker_symbol.lower()}.csv'
    df = pd.read_csv(csv_name, parse_dates=['Date_add'])
    df.rename(columns={'Date_add': 'Date'}, inplace=True)
    df['Date'] = pd.to_datetime(df['Date'], utc=True).dt.tz_convert(None)

    # Filter for Tuesdays
    df['Day of Week'] = df['Date'].dt.day_name()
    tuesdays_df = df[df['Day of Week'] == 'Tuesday'].copy()

    # Calculate daily average price
    tuesdays_df.loc[:, 'Avg Price'] = (tuesdays_df['Open'] + tuesdays_df['Close']) / 2

    # Simulate DCA purchases
    tuesdays_df.loc[:, 'Shares Purchased'] = investment_amount / tuesdays_df['Avg Price']
    tuesdays_df.loc[:, 'Cumulative Shares'] = tuesdays_df['Shares Purchased'].cumsum()

    # Calculate the market value of the cumulative shares
    tuesdays_df.loc[:, 'Market Value'] = tuesdays_df['Cumulative Shares'] * tuesdays_df['Close']

    # Calculate the cumulative invested amount
    tuesdays_df.loc[:, 'Cumulative Invested'] = investment_amount * (tuesdays_df.reset_index().index + 1)

    # Plot the DCA strategy results
    fig, ax = plt.subplots(figsize=(12, 8))

    ax.plot(tuesdays_df['Date'], tuesdays_df['Cumulative Invested'], label='Cumulative Invested', color='blue')
    ax.plot(tuesdays_df['Date'], tuesdays_df['Market Value'], label=f'Market Value ({ticker_symbol} Close)', color='green')
    ax.set_title(f'Dollar-Cost Averaging Strategy: Cumulative Invested Amount vs. Market Value Over Time ($25 Weekly) - {ticker_symbol}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Value ($)')
    ax.legend()
    ax.grid(True)

    plt.show()

    # Display the final few rows of the DataFrame for inspection
    # display(tuesdays_df.tail())